In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Đọc dữ liệu từ file train_data, hiển thị thông tin cần thiết.

In [ ]:
dataTrain_titanic = pd.read_csv('D:/Hocmay/Titanic_ML_From_Disaster/Titanic_Project_Information_Data/train.csv')
dataTrain_titanic

In [ ]:
dataTrain_titanic.head()

Convert cột PassengerId của Data Train thành cột index trong DataFrame.

In [ ]:
dataTrain_titanic.set_index('PassengerId', inplace=True)
dataTrain_titanic.head()

Ở file train_data, xuất hiện cột "Survived" là cột cần dự đoán, còn ở file test_data, không có cột "Survived".

In [ ]:
dataTest_titanic = pd.read_csv('D:/Hocmay/Titanic_ML_From_Disaster/Titanic_Project_Information_Data/test.csv')
dataTest_titanic

In [ ]:
dataTest_titanic.head()

Convert cột PassengerId của Data Test thành cột index trong DataFrame.

In [ ]:
dataTest_titanic.set_index('PassengerId', inplace=True)
dataTest_titanic.head()

Tạo ra 1 bảng thống kê tóm tắt các cột số trong dataframe. Bảng thống kê này bao gồm các thông tin sau:
- count: số lượng giá trị không bị thiếu
- mean: giá trị trung bình
- std: độ lệch chuẩn
- min: giá trị nhỏ nhất
- 25%: phần centile 25
- 50%: phần centile 50
- 75%: phần centile 75
- max: giá trị lớn nhất

[4]:    
**SibSp** ranges from 0 to 8, so it is divided into the following stages:
- Group 1: SibSp = 0
- Group 2: SibSp = 1
- Group 3: SibSp = 2
- Group 4: SibSp = 3
- Group 5: SibSp = 4
- Group 6: SibSp = 5
- Group 7: SibSp = 8

In the training data, SibSp values of 6 and 7 do not appear, so they will not be considered, and the maximum value displayed is 8.

**Parch** ranges from 0 to 6, so it is divided into the following stages:
- Group 1: Parch = 0
- Group 2: Parch = 1
- Group 3: Parch = 2
- Group 4: Parch = 3
- Group 5: Parch = 4
- Group 6: Parch = 5
- Group 7: Parch = 6

In the training data, Parch values greater than 6 do not appear, so they will not be considered, and the maximum value displayed is 6.


In [ ]:
dataTrain_titanic.describe()

Display the correlation heatmap for the groups above.

In [ ]:
import seaborn as sns

sns.heatmap(dataTrain_titanic.corr(numeric_only=True), cmap="inferno", annot=True)
plt.show()

Because the new version of the Seaborn library no longer supports writing `sns.heatmap(dataTrain_titanic.corr(), cmap="inferno", annot=True)` in this way in the `sns.heatmap` function, the solution is to pass a boolean parameter into the `heatmap` function, `sns.heatmap(dataTrain_titanic.corr(numeric_only=True), cmap="inferno", annot=True)`. If `True`, the value will be displayed on the heatmap, and if `False`, the value will not be displayed. https://seaborn.pydata.org/generated/seaborn.heatmap.html

Method "stratified random split of training data" đảm bảo rằng mỗi tập con của dữ liệu huấn luyện (sau khi chia) có sự phân bổ tương tự như tổng thể dữ liệu ban đầu đối với một số đặc tính nhất định (ví dụ: tỷ lệ của các phòng).

Sử dụng 80% dữ liệu để huấn luyện và 20% dữ liệu để kiểm tra.

In [10]:
dataTrain_titanic = dataTrain_titanic.reset_index(drop=True)

Reset lại chỉ số của DataFrame dataTrain_titanic trước khi chia dữ liệu, để các chỉ số của DataFrame là liên tục và bắt đầu từ 0.

In [11]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_indices, test_indices in split.split(dataTrain_titanic, dataTrain_titanic[["Survived", "Pclass", "Sex"]]):
    strat_train_set = dataTrain_titanic.loc[train_indices]
    strat_test_set = dataTrain_titanic.loc[test_indices]

In [ ]:
strat_test_set

Vẽ biểu đồ Histogram của các nhóm trên dữ liệu huấn luyện và dữ liệu kiểm tra.


In [ ]:
plt.subplot(1, 2, 1)
strat_train_set["Survived"].hist()
strat_train_set["Pclass"].hist()

plt.subplot(1, 2, 2)
strat_test_set["Survived"].hist()
strat_test_set["Pclass"].hist()

plt.show()

Tạo ra 4 biểu đồ histogram để so sánh phân phối các cột Survived, Pclass giữa dữ liệu huấn luyện và dữ liệu kiểm tra.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
strat_train_set["Survived"].hist()
plt.title("Training Set - Survived")

plt.subplot(2, 2, 2)
strat_train_set["Pclass"].hist()
plt.title("Training Set - Pclass")

plt.subplot(2, 2, 3)
strat_test_set["Survived"].hist()
plt.title("Test Set - Survived")

plt.subplot(2, 2, 4)
strat_test_set["Pclass"].hist()
plt.title("Test Set - Pclass")

plt.tight_layout()
plt.show()

Phân ra categorical và numerical features để nhận biết kiểu dữ liệu phục vụ cho việc xử lý giá trị bị thiếu và huấn luyện mô hình.

Categorical Features - Đặc điểm phân loại:
- nominal: không có thứ tự, không thể so sánh với nhau.
- ordinal: có thứ tự, có thể so sánh với nhau.
- ratio: có thứ tự, có thể so sánh với nhau, có giá trị 0.
- interval: có thứ tự, có thể so sánh với nhau, không có giá trị 0.

Numerical Features - Đặc điểm số:
- discrete: giá trị rời rạc.
- continuous: giá trị liên tục.
- time-series: giá trị thời gian.

In [ ]:
dataTrain_titanic.info()

Kết quả cho thấy: Data columns (total 12 columns): có 12 cột dữ liệu, chi tiết như sau:
- Survived: sống sót hay không.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: int64 (kiểu dữ liệu số nguyên).
  - unique: 2 (có 2 giá trị duy nhất).
- Pclass: hạng ghế.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: int64 (kiểu dữ liệu số nguyên).
  - unique: 3 (có 3 giá trị duy nhất).
- Name: tên hành khách.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: object (kiểu dữ liệu chuỗi).
- Sex: giới tính.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: object (kiểu dữ liệu chuỗi).
  - unique: 2 (có 2 giá trị duy nhất).
- Age: tuổi.
  - non-null: 714 (có 177 giá trị bị thiếu).
  - data type: float64 (kiểu dữ liệu số thực).
  - mean: 29.7 (giá trị trung bình).
- SibSp: số lượng anh chị em hoặc vợ chồng đi cùng.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: int64 (kiểu dữ liệu số nguyên).
  - unique: 7 (có 7 giá trị duy nhất).
- Parch: số lượng cha mẹ hoặc con cái đi cùng.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: int64 (kiểu dữ liệu số nguyên).
  - unique: 7 (có 7 giá trị duy nhất).
- Ticket: số vé.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: object (kiểu dữ liệu chuỗi).
- Fare: giá vé.
  - non-null: 891 (không có giá trị bị thiếu).
  - data type: float64 (kiểu dữ liệu số thực).
  - mean: 32.2 (giá trị trung bình).
- Cabin: số phòng.
  - non-null: 204 (có 687 giá trị bị thiếu).
  - data type: object (kiểu dữ liệu chuỗi).
Embarked: cảng lên tàu.
  - non-null: 889 (có 2 giá trị bị thiếu).
  - data type: object (kiểu dữ liệu chuỗi).
  - unique: 3 (có 3 giá trị duy nhất). 

Age, Cabin, Embarked là 3 giá trị bị missing value -> cần xử lý missing value.

Giải thích loại Data Type của các mục dữ liệu:
- Survived có 2 giá trị duy nhất là 0 và 1:
  - 0: không sống sót.
  - 1: sống sót.
  - Nên Survived là kiểu dữ liệu số nguyên.
- Pclass có 3 giá trị duy nhất là 1, 2 và 3:
  - 1: phòng hạng 1.
  - 2: phòng hạng 2.
  - 3: phòng hạng 3.
  - Nên Pclass là kiểu dữ liệu số nguyên.
- Name là chuỗi ký tự, không thể so sánh
  - Nên Name là kiểu dữ liệu chuỗi.
- Sex có 2 giá trị duy nhất:
  - male: giới tính nam.
  - female: giới tính nữ.
  - Nên Sex là kiểu dữ liệu chuỗi.
- Age là số thực, có giá trị trung bình là 29.7.
  - Nên Age là kiểu dữ liệu số thực.
- SibSp có 7 giá trị (1 ace/vochong, 2 ace/vochong,... 8 ace/vochong):
  - Nên SibSp là kiểu dữ liệu số nguyên.
- Parch có 7 giá trị (1 con, 2 con,... 6 con):
  - Nên Parch là kiểu dữ liệu số nguyên.
- Ticket là chuỗi ký tự, không thể so sánh
  - Nên Ticket là kiểu dữ liệu chuỗi.
- Fare là giá vé tàu:
  - Nên Fare là kiểu dữ liệu số thực.
- Cabin là chuỗi ký tự, không thể so sánh
  - Nên Cabin là kiểu dữ liệu chuỗi.
- Embarked có 3 giá trị duy nhất là S, C và Q:
  - S: Southampton.
  - C: Cherbourg.
  - Q: Queenstown.
  - Nên Embarked là kiểu dữ liệu chuỗi.

In [ ]:
strat_train_set.info()

Kết quả cho thấy: tập dữ liệu train có
Index: 712 hàng, mỗi hàng đại diện cho một hành khách trên tàu Titanic.
Data columns (total 12 columns): có 12 cột dữ liệu, chi tiêt như sau:
- PassengerId: số thứ tự hành khách.
  - non-null: 712 (không có giá trị thiếu)
  - dtype: int64 (kiểu dữ liệu là số nguyên)
- Survived: số người sống sót
  - non-null: 712 (không có giá trị thiếu)
  - dtype: int64 (kiểu dữ liệu là số nguyên)
- Pclass: hạng vé
  - non-null: 712 (không có giá trị thiếu)
  - dtype: int64 (kiểu dữ liệu là số nguyên)
- Name: tên hành khách
  - non-null: 712 (không có giá trị thiếu)
  - dtype: object (kiểu dữ liệu là chuỗi)
- Sex: giới tính
  - non-null: 712 (không có giá trị thiếu)
  - dtype: object (kiểu dữ liệu là chuỗi)
- Age: tuổi
  - non-null: 568 (có giá trị thiếu)
  - dtype: float64 (kiểu dữ liệu là số thực)
  - note: data train có tổng cộng 712 hành khách, trong đó có 568 hành khách có thông tin về tuổi và 144 hành khách không có thông tin về tuổi.
- SibSp: số lượng anh chị em hoặc vợ chồng đi cùng
  - non-null: 712 (không có giá trị thiếu)
  - dtype: int64 (kiểu dữ liệu là số nguyên)
- Parch: số lượng bố mẹ hoặc con cái đi cùng
  - non-null: 712 (không có giá trị thiếu)
  - dtype: int64 (kiểu dữ liệu là số nguyên)
- Ticket: số vé
  - non-null: 712 (không có giá trị thiếu)
  - dtype: object (kiểu dữ liệu là chuỗi)
- Fare: giá vé
  - non-null: 712 (không có giá trị thiếu)
  - dtype: float64 (kiểu dữ liệu là số thực)
- Cabin: số phòng
  - non-null: 161 (có giá trị thiếu)
  - dtype: object (kiểu dữ liệu là chuỗi)
  - note: data train có tổng cộng 712 hành khách, trong đó có 161 hành khách có thông tin về số phòng và 551 hành khách không có thông tin về số phòng.
- Embarked: cảng lên tàu
  - non-null: 710 (có giá trị thiếu)
  - dtype: object (kiểu dữ liệu là chuỗi)
  - note: data train có tổng cộng 712 hành khách, trong đó có 710 hành khách có thông tin về cảng lên tàu và 2 hành khách không có thông tin về cảng lên tàu.

In [ ]:
dataTest_titanic.info()

Categorical: Survived, Sex, Embarked, Pclass(ordinal), SibSp, Parch
Numerical(continuous): Age, Fare
Mix types of data: Ticket, Cabin
Contain errors or typos: Name
Blank, null or empty values: Age > Cabin > Embarked
Various Data Types: float64, int64, object(String)  (0 - 1: categorical)

In [ ]:
strat_test_set.info()

Ép kiểu dữ liệu của cột.

In [20]:
dataTrain_titanic["Survived"] = dataTrain_titanic["Survived"].astype('category')

Check kiểu dữ liệu cột Survived, nếu kiểu dữ liệu là int64 thì chuyển thành category.

In [ ]:
dataTrain_titanic["Survived"].dtype

Check lại data Train sau khi chuyển kiểu dữ liệu của cột Survived.

In [ ]:
dataTrain_titanic.info()

tạo Function để chuyển kiểu dữ liệu của cột từ các kiểu dữ liệu int, object, float sang category để: giảm dung lượng bộ nhớ, tăng tốc độ xử lý dữ liệu, dễ dàng xử lý các giá trị phân loại -> trực quan hóa dữ liệu và giảm thời gian huấn luyện mô hình.

In [24]:
converted_dataTrain = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]
def convert_cat(df, converted_dataTrain):
    for convert in converted_dataTrain:
        df[convert] = df[convert].astype('category') # 1
        # df.convert = df.converted_dataTrain.astype('category') # 2
        
convert_cat(dataTrain_titanic, converted_dataTrain) # dataTrain_titanic.convert_cat(converted_dataTrain)
convert_cat(dataTest_titanic, converted_dataTrain) # dataTest_titanic.convert_cat(converted_dataTrain)

các kiểu dữ liệu nào thuộc category hoặc object thì chuyển sang category -> giúp trực quan hóa dữ liệu hơn.

In [ ]:
dataTrain_titanic.info()
dataTest_titanic.info()

Ước lượng tỷ lệ sống sót trên tàu Titanic.

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin